In [ ]:
import os
import glob
import trimesh
import numpy as np
import tensorflow as tf
from tensorflow import data as tf_data
from tensorflow.python.framework import ops
import keras
from keras import layers
from sklearn.model_selection import train_test_split
import sklearn.utils
from matplotlib import pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler, MinMaxScaler

keras.utils.set_random_seed(seed=42)

import sys
sys.path.append('/Users/author/')
from Utils_functions import load_and_preprocess_naca_data, SimpleBaseModel, kfold_cross_validation_MLP, PointTransformerModel, kfold_cross_validation_PT

2025-03-10 15:50:33.598372: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
BATCH_SIZE = 16
TRAIN_SIZE = 0.70
TEST_SIZE = 0.15
NUM_TARGETS = 3  

#C-PROP
points_cluster, labels_cluster, train_ds_cluster, val_ds_cluster, test_ds_cluster, NUM_POINTS, NUM_FEATURES = load_and_preprocess_naca_data('data/points_prop_zenodo.npy','data/labels_zenodo.npy', 
                                                                                                                              BATCH_SIZE, TRAIN_SIZE, TEST_SIZE, augment=False, augment_factor=3, scale= True)

# Schillaci et al.
points_schilla, labels_schilla, train_ds_schilla, val_ds_schilla, test_ds_schilla, NUM_POINTS_SCHILLA, NUM_FEATURES_SCHILLA = load_and_preprocess_naca_data('data/lines_NACA_schilla_weight_complete.npy','data/labels_NACA_schilla_weight_complete.npy', 
                                                                                                                                BATCH_SIZE, TRAIN_SIZE, TEST_SIZE, augment=False, augment_factor=3, scale= True, only_p=False) # seto only p=true for CR+CR

In [ ]:

base_model_schilla = SimpleBaseModel(NUM_POINTS_SCHILLA, NUM_FEATURES_SCHILLA, NUM_TARGETS)

history_schilla = base_model_schilla.train(train_ds_schilla, val_ds_schilla, epochs=1000, patience=10)

base_model_schilla.evaluate(test_ds_schilla)

base_model_schilla.plot_training_history(history_schilla)


In [ ]:

base_model_cluster = SimpleBaseModel(NUM_POINTS, NUM_FEATURES, NUM_TARGETS)

history_cluster = base_model_cluster.train(train_ds_cluster, val_ds_cluster, epochs=1000, patience=10)

base_model_cluster.evaluate(test_ds_cluster)

base_model_cluster.plot_training_history(history_cluster)

In [ ]:

fig, axs = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle("True vs Predicted Values for Digits (Comparison of Models)", fontsize=16)

base_model_schilla.show_predictions(
    test_ds=test_ds_schilla,
    ax=axs[0],
    title="Model Schilla Predictions",
    model_name="Model Schilla"
)

base_model_cluster.show_predictions(
    test_ds=test_ds_cluster,
    ax=axs[1],
    title="Model Cluster Predictions",
    model_name="Model Cluster"
)

plt.tight_layout(rect=[0, 0, 1, 0.95]) 
plt.show()


In [ ]:

mae, max_error, max_error_position = base_model_schilla.evaluate_relative_error(test_ds_schilla)

# Print summary
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Maximum Error: {max_error:.4f}")
#print(f"Position of Maximum Error: {max_error_position}")

accuracy, correct_predictions = base_model_schilla.evaluate_accuracy(test_ds_schilla)

In [ ]:
# Evaluate relative error
mae, max_error, max_error_position = base_model_cluster.evaluate_relative_error(test_ds_cluster)

# Print summary
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Maximum Error: {max_error:.4f}")
#print(f"Position of Maximum Error: {max_error_position}")

# Evaluate accuracy
accuracy, correct_predictions = base_model_cluster.evaluate_accuracy(test_ds_cluster)

In [ ]:
kfold_cross_validation_MLP(points_schilla, labels_schilla, NUM_POINTS_SCHILLA,
                            NUM_FEATURES_SCHILLA, NUM_TARGETS, n_splits=5, batch_size=16)

In [ ]:
# Calcolo dell'Accuracy: arrotondamento dei valori predetti ai valori interi
y_true_schilla = np.concatenate([y for x, y in test_ds_schilla], axis=0)  # Label reali dal test set
y_pred_schilla = base_model_schilla.predict(test_ds_schilla)  # Predizioni del modello sul test set

# Arrotonda i valori
y_pred_rounded_schilla= np.round(y_pred_schilla, decimals=0)  # Arrotonda le predizioni ai valori interi

# Calcola l'accuracy confrontando le predizioni arrotondate con i label arrotondati
accuracy_schilla = np.mean(np.all(y_true_schilla == y_pred_rounded_schilla, axis=1))
accuracy_schilla_1 = np.mean(y_true_schilla[:,0] == y_pred_rounded_schilla[:,0])
accuracy_schilla_2 = np.mean(y_true_schilla[:,1] == y_pred_rounded_schilla[:,1])
accuracy_schilla_3 = np.mean(y_true_schilla[:,2] == y_pred_rounded_schilla[:,2])
beccati= np.sum(np.all(y_true_schilla == y_pred_rounded_schilla, axis=1))
print(f"Beccati: {beccati}/{np.shape(y_true_schilla)[0]}")
print(f"Test Accuracy (Classificazione): {accuracy_schilla*100:.2f}%\n")
print(f"Test Accuracy I cifra: {accuracy_schilla_1*100:.2f}%")
print(f"Test Accuracy II cifra: {accuracy_schilla_2*100:.2f}%")
print(f"Test Accuracy III cifra: {accuracy_schilla_3*100:.2f}%\n")

errors = np.abs(y_true_schilla - y_pred_schilla)
mae = np.mean(errors)
max_error = np.max(errors)
max_error_position = np.unravel_index(np.argmax(errors), errors.shape)

mae_1 = np.mean(errors[:, 0])
mae_2 = np.mean(errors[:, 1])
mae_3 = np.mean(errors[:, 2])

print(f"Errore relativo Medio Perc. sulla I cifra (MAE): {mae_1:.4f}, {mae_1 / 9 * 100:.2f}%")
print(f"Errore relativo Medio Perc. sulla II cifra (MAE): {mae_2:.4f}, {mae_2 / 9 * 100:.2f}%")
print(f"Errore relativo Medio Perc. sulla III cifra (MAE): {mae_3:.4f}, {mae_3 / 45 * 100:.2f}%\n")

print(f"Errore Assoluto Medio sulla I cifra (MAE): {mae_1:.4f}, {mae_1:.3f}")
print(f"Errore Assoluto Medio sulla II cifra (MAE): {mae_2:.4f}, {mae_2:.3f}")
print(f"Errore Assoluto Medio sulla III cifra (MAE): {mae_3:.4f}, {mae_3:.3f}\n")

print(f"Errore Medio assoluto (MAE): {mae:.4f}")
print(f"Errore Massimo: {max_error:.4f} in posizione {max_error_position}")

In [ ]:

errors = np.abs(y_true_schilla[:, 0] - y_pred_schilla[:, 0])

mean_error = np.mean(errors)

plt.hist(errors, bins=50, alpha=0.7, label="Absolute Errors")
plt.axvline(mean_error, color='r', linestyle='dashed', linewidth=2, label=f"Mean Error: {mean_error:.3f}")

plt.xlabel("Absolute Error")
plt.ylabel("Frequency")
plt.title("Distribution of Absolute Errors")
plt.legend()
plt.show()


In [13]:
errors = np.abs(y_true_schilla[:,2] - y_pred_schilla[:,2])
print(f"Percentuale di errori tra 0.45 e 0.5: {np.mean((errors > 0.45) & (errors <= 0.5)) * 100:.2f}%")


Percentuale di errori tra 0.45 e 0.5: 1.61%


In [ ]:
# C-FREE
points_free, labels_free, train_ds_free, val_ds_free, test_ds_free, NUM_POINTS, NUM_FEATURES = load_and_preprocess_naca_data('data/points_zenodo.npy','data/labels_zenodo.npy', 
                                                                                                                              BATCH_SIZE, TRAIN_SIZE, TEST_SIZE, augment=False, augment_factor=3, scale= True)

In [ ]:

Pt = PointTransformerModel(NUM_POINTS, NUM_FEATURES, NUM_TARGETS, invariant = False)

history_Pt = Pt.train(train_ds_free, val_ds_free, epochs=1000, patience=10)


Pt.evaluate(test_ds_free)

Pt.plot_training_history(history_Pt)

In [ ]:
kfold_cross_validation_PT(points_free, labels_free, NUM_POINTS,
                            NUM_FEATURES, NUM_TARGETS, n_splits=5, batch_size=16)